In [1]:
import pandas as pd
from sp_client import Spotify_Client
from cluster_classifier import predict
import hashlib
import pickle

In [7]:
recent_df  = pd.read_csv("data/recent.csv")
all_df = pd.read_csv("data/all_songs_clustered.csv", index_col = 'uri')

In [6]:
recent_df[::-1].reset_index(drop = True).to_csv("data/recent.csv", index = False)

In [ ]:
with open('data/timestamp.data', 'rb') as f:
    ts = pickle.load(f)

In [14]:
sp = Spotify_Client()

In [15]:
sp.current_user_recently_played(limit = 1)['items'][0]['track']['name']

'Henny in the Hamptons'

In [ ]:
ts

In [ ]:
with open('data/last_session.data', 'wb') as f:
    pickle.dump({'timestamp':ts, 'session':'d2a04f014f6d12fcb72e387150666fbddfc3dae5'}, f)

In [ ]:
with open('temp.data', 'rb') as f:
    data = pickle.load(f)

In [ ]:
data

In [ ]:
sp = Spotify_Client()

In [ ]:
recent_df.head()

In [ ]:
errors = []
for i,r in recent_df.iterrows():
    try:
        recent_df.loc[i, 'danceability'] = float(r['danceability'])
    except ValueError:
        errors.append(i)
        print(i)
        

In [ ]:
len(errors)

In [ ]:
list(filter(lambda i: recent_df.loc[i,'uri'] not in all_df.index, errors))

In [ ]:
cols = recent_df.columns[5:-1]

In [ ]:
cols

In [ ]:
sp.audio_features('spotify:track:5HBDBBr5OV90lubqW8ctJF')

In [ ]:
for i in errors:
    uri = recent_df.loc[i, 'uri']
    af = sp.audio_features(uri)[0]
    d = {k:v for (k,v) in af.items() if k in cols}
    recent_df.loc[i, cols] = d
    recent_df.loc[i, 'label'] = predict(list(d.values()))
    
    

In [ ]:
cols

In [ ]:
for c in list(cols) + ['label']:
    recent_df.loc[:, c] = pd.to_numeric(recent_df.loc[:,c])

In [ ]:
recent_df.to_csv("data/recent.csv", index = False)

In [ ]:
recent_df.dtypes

In [ ]:
recent_df.loc[:, cols] = pd.to_numeric(recent_df.loc[:, cols])

In [ ]:
recent_df.loc[errors, ]

In [ ]:

predict(recent_df.loc[errors[0], cols])

In [ ]:
#update labels

In [ ]:
af = sp.audio_features(uri)[0]


In [ ]:
recent_df.loc[errors, :]

In [ ]:
int('uri')

In [ ]:
len({hashlib.sha1(str(x).encode('utf-8')).hexdigest() for x in recent_df.session.unique()})

In [ ]:
rdf = recent_df.copy()
for s in recent_df.session.unique():
    rdf.loc[recent_df.session == s, 'session'] = hashlib.sha1(str(s).encode('utf-8')).hexdigest()
    

In [ ]:
rdf.to_csv("data/recent.csv", index = False)

In [ ]:
rdf.head(50)

In [ ]:
int([])

In [ ]:
rdf.loc[0,'danceability']

In [ ]:
for i,r in rdf.iterrows():
    if type(r['danceability'].value)

In [ ]:
try:
    rdf['danceability'] = pd.to_numeric(rdf['danceability']) 
except ValueError:
    # I want to register on my log the message recived on ORIGINAL VALUE
    mask = pd.to_numeric(rdf['danceability'].fillna('0'), errors='coerce').isna() 
    L = rdf.loc[mask, 'danceability'].tolist()
    for val in L:
        print (f"Not converted values are: {val}")

In [ ]:
rdf.isna().any()

In [ ]:
all_df.head()

In [ ]:
##restructure recent.csv to include audio features + label

# cols =['danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
#        'liveness', 'valence', 'tempo', 'label']
# d = {c:[] for c in cols}
# for i,row in recent_df.iterrows():
#     uri = row['uri']
#     try:
#         af = all_df.loc[uri, cols]
#     except KeyError:
#         try: 
#             af = all_df.loc[(all_df.artist == row.artist) & (all_df.title == row.title), cols]
#         except:
#             print(i)
#     for c in cols:
#         d[c].append(af[c])
# assert len(recent_df) == len(d['danceability'])
# for col in d:
#     recent_df[col] = pd.Series(d[col])
# recent_df.to_csv("data/recent.csv", index = False)

In [ ]:
def get_start_rows(df):
    starts = []
    grouped = df.groupby('session')
    [starts.append(grouped.get_group(k).index.values.astype(int)[0]) for k,_ in grouped]
    return sorted(starts)


In [ ]:
get_start_rows(df)

In [ ]:
df.join(all_df, on = 'uri')

In [ ]:
df